In [23]:
%reload_ext autoreload
%autoreload 2
# %matplotlib inline

In [24]:
from utils import *

In [25]:
import altair as alt
import pandas as pd

In [26]:
import arviz as az
from causalgraphicalmodels import CausalGraphicalModel
import daft
import matplotlib.pyplot as plt
import numpy as onp
import pandas as pd

from jax import lax
import jax.numpy as np
from jax.random import PRNGKey

import numpyro
from numpyro.contrib.autoguide import (AutoContinuousELBO,
                                       AutoLaplaceApproximation)
from numpyro.diagnostics import print_summary
import numpyro.distributions as dist
from numpyro.infer import Predictive, SVI
import numpyro.optim as optim

%config InlineBackend.figure_formats = ["svg"]
az.style.use("arviz-darkgrid")

In [29]:
# load data and copy
WaffleDivorce = pd.read_csv("../data/WaffleDivorce.csv", sep=";")
d = WaffleDivorce

# standardize variables
d["A"] = d.MedianAgeMarriage.pipe(lambda x: (x - x.mean()) / x.std())
d["D"] = d.Divorce.pipe(lambda x: (x - x.mean()) / x.std())
d["M"] = d.Marriage.pipe(lambda x: (x - x.mean()) / x.std())

In [28]:
import xarray as xr
xr.set_options(display_style="html")

In [22]:
data = np.random.rand(2,3,4)
da1 = xr.DataArray(data, coords = {'x':[10,20], 'y':[8,9,10], 'z':list('abcd')}, dims=list('xyz'), name="George"  )
da2 = xr.DataArray(data**2, coords = {'i':[10,20], 'j':[8,9,10], 'z':list('abcd')}, dims=list('ijz'), name="Pete"  )

xr.Dataset({'g': da1, 'p':da2})

<xarray.Dataset>
Dimensions:  (i: 2, j: 3, x: 2, y: 3, z: 4)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 8 9 10
  * z        (z) <U1 'a' 'b' 'c' 'd'
  * i        (i) int64 10 20
  * j        (j) int64 8 9 10
Data variables:
    g        (x, y, z) float64 0.8748 0.8168 0.3182 ... 0.1027 0.8412 0.4888
    p        (i, j, z) float64 0.7653 0.6672 0.1013 ... 0.01055 0.7076 0.239

In [40]:
ds = xr.Dataset( d.loc[:,['Location','A','D','M']] )
ds

<xarray.Dataset>
Dimensions:   (dim_0: 50)
Coordinates:
  * dim_0     (dim_0) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
Data variables:
    Location  (dim_0) object 'Alabama' 'Alaska' ... 'Wisconsin' 'Wyoming'
    A         (dim_0) float64 -0.6063 -0.6867 -0.2042 ... -0.8475 0.1978 -1.491
    D         (dim_0) float64 1.654 1.544 0.6107 2.094 ... 0.6656 -0.7623 0.3361
    M         (dim_0) float64 0.02264 1.55 0.04897 ... 0.5493 -0.7673 2.787